In [7]:
import pandas as pd
import numpy as np
import scipy.stats as sps

inputCauchyData = pd.read_csv('cauchy.txt');
selection = np.array(inputCauchyData)
N = len(selection)        

our_x0_span = np.arange(-1000, 1000.01, 0.01);

# Параметр масштаба тета - единичный.
theta = 1

# Ищем мы параметр сдвига - x0
theBestX0 = our_x0_span[0];

# Максимум логарифма функции правдоподобия.
ourMax = -np.inf;

In [8]:
# Для половины выборки
for x0 in our_x0_span:
    cauchy_x0 = sps.cauchy(x0, theta);
    candidate = np.sum(cauchy_x0.logpdf(selection[:N/2]));
    if (candidate > ourMax) :
        ourMax = candidate;
        theBestX0 = x0;
        
print (theBestX0)
print(selection.mean())

-212.470000001
-212.424257813


/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Результат близкий к среднему значению, т.к. Распределение Коши симметрично относительно x0. X0 - матожидание.

In [9]:
# Теперь для всей выборки.
for x0 in our_x0_span:
    cauchy_x0 = sps.cauchy(x0, theta);
    candidate = np.sum(cauchy_x0.logpdf(selection[:N]));
    if (candidate > ourMax) :
        ourMax = candidate;
        theBestX0 = x0;
        
print (theBestX0)
print(selection.mean())

-212.470000001
-212.424257813


Идентичный результат для всей выборки. Можно сделать вывод, что достаточно смотреть на половину, чтобы достаточно точно оценить x0.

In [10]:
class weibull_gen(sps.rv_continuous):
    def _cdf(self, x, gamma):
        return 1 - np.exp(-x ** gamma)
    
weibull = weibull_gen(a=0, name="weibull")

inputWeibullData = pd.read_csv('Weibull.txt')
selection = np.array(inputWeibullData);

ourLeftEdge = -2;
ourRightEdge = 2;
log_gamma_span = np.arange(-2, 2, 0.001);

neededGamma = -np.inf;
local_max = -np.inf;

In [11]:
# По четырём годам
N = 4 * 365;
for log_gamma in log_gamma_span :
    gamma = log_gamma ** 10;
    lg_lh_foo = np.sum(weibull.logpdf(selection[:N], gamma));
    if (local_max < lg_lh_foo) :
        local_max = lg_lh_foo;
        neededGamma = gamma;
print(np.log10(neededGamma));

1.71433900943


Log(gamma) основываясь на 4 годах

In [12]:
# По за всё время.
N = 10 * 365;
for log_gamma in log_gamma_span :
    gamma = log_gamma ** 10;
    lg_lh_foo = np.sum(weibull.logpdf(selection[:N], gamma));
    if (local_max < lg_lh_foo) :
        local_max = lg_lh_foo;
        neededGamma = gamma;
print(np.log10(neededGamma));

1.71726453653


Полученное значение log(gamma) с 40 процентов выборки обладает высокой точностью. Значит чтобы посчитать его, не нужно даже смотреть на всю выборку.